需要import的lib都放在這~

In [21]:
# import library
import pandas as pd
import numpy as np
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import plotly as py
import math
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [22]:
"""Helper of data_mining from TA
Helper functions for data mining lab session 2018 Fall Semester
Author: Elvis Saravia
Email: ellfae@gmail.com
"""

def format_rows(docs):
    """ format the text field and strip special characters """
    D = []
    for d in docs.data:
        temp_d = " ".join(d.split("\n")).strip('\n\t')
        D.append([temp_d])
    return D

def format_labels(target, docs):
    """ format the labels """
    return docs.target_names[target]

def check_missing_values(col):
    """ functions that check and verifies if there are missing values in dataframe """
    counter = 0
    for element in col:
        if element == True:
            counter+=1
    return ("The amount of missing records is: ", counter)

def tokenize_text(text, remove_stopwords=False):
    """
    Tokenize text using the nltk library
    """
    tokens = []
    for d in nltk.sent_tokenize(text, language='english'):
        """split a text to sentences"""
        for word in nltk.word_tokenize(d, language='english'):
            "split sentence to words"
            # filters here
            tokens.append(word)
    return tokens

In [23]:
""" Prepare dataset
    這個dataset是從3個網站中取得評論以及分數,分數有0跟1兩種
"""
website = ['amazon_cells', 'imdb', 'yelp']

準備我們要使用的dataset

In [24]:
"""Transfer data to a pandas Dataframe"""
import pandas as pd

now_dir = os.getcwd()
print("now_dir:{}".format(now_dir))
comments = []
webs = []
scores = []
for web in website:
    fileName = now_dir + "/../dataset/sentiment labelled sentences/"
    fileName += web + "_labelled.txt"
    print(fileName)
    with open(fileName,encoding="utf-8") as f:
        for line in f.readlines():
            """用這一行可以將input的字串用指定的子字串分割"""
            # print(line.split(" "))
            """先使用這個分割,就可以在print之後看到諸如\t, \n這樣的字元出現"""
            """如此一來我們就可以知道我們實際上該用甚麼子字串來分割"""

            """用\t就可以將文字分成評論跟分數兩個"""
            seperate_str = line.split("\t")
            comment = seperate_str[0]
            score = seperate_str[1].strip('\n')
            comments.append(comment)
            scores.append(score)
            webs.append(web)

            """line.split(".") 用.將line分割開來"""
            """ "".join() 將上一行分割後輸出的list合併成一個新的string, 使用"" 作為合併時插入的間隔"""
            """ .strip() 再將上一行輸出的string將arg中輸入的字元,從input的頭或尾去除"""
            # print("".join(line.split(".")).strip('\n'))

            """切割好資料了,現在將他們放進一個dataFrame當中"""

        print("---" * 10)
data = pd.DataFrame(list(zip(webs, comments,scores)),columns =['Web', 'Comment','Score'])


now_dir:d:\學校資料\111上學期\資料探勘與應用\python_env\data_mining\src
d:\學校資料\111上學期\資料探勘與應用\python_env\data_mining\src/../dataset/sentiment labelled sentences/amazon_cells_labelled.txt
------------------------------
d:\學校資料\111上學期\資料探勘與應用\python_env\data_mining\src/../dataset/sentiment labelled sentences/imdb_labelled.txt
------------------------------
d:\學校資料\111上學期\資料探勘與應用\python_env\data_mining\src/../dataset/sentiment labelled sentences/yelp_labelled.txt
------------------------------


1. Meaningful visulaization

In [25]:
"""這部分做的跟part1、2相同，我就不贅述了"""

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
data_counts = count_vect.fit_transform(data.Comment) 
#learn the vocabulary and return document-term matrix
print(data_counts.shape)

term_frequencies = []
for j in range(0,data_counts.shape[1]):
    term_frequencies.append(sum(data_counts[:,j].toarray()))
term_frequencies = np.asarray(data_counts.sum(axis=0))[0]
total_frequency = sum(term_frequencies.tolist())
mean_frequency = total_frequency / len(term_frequencies)
mean_frequency

target_id = []
target_freq = []
for index,freq in enumerate(term_frequencies):
    if freq > mean_frequency:
        target_id.append(index)
        target_freq.append(freq)


import plotly.express as px
import pandas as pd

fig = px.bar(   
                target_freq,
                x=count_vect.get_feature_names_out()[target_id],
                y=target_freq
            )
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

(3000, 5155)


2. TF-IDF

In [29]:
"""建立TD-IDF"""

import seaborn as sns

term_frequencies = []
for i in range(data_counts.shape[0]):
    term_freq = []
    # 找出第i個comment當中包含甚麼詞
    _, indexs = data_counts[i].nonzero()
    # 含有的詞的數量
    numWords = len(indexs)
    for ind in indexs:
        # 出現的次數 / 整個句子有幾個詞 = freq
        term_freq.append([ind, data_counts[i][0, ind] / numWords])
    term_frequencies.append(term_freq)

inverse_document_freqs = []
n = data_counts.shape[1]
print(n)
for i in range(n):
    # 這個詞出現在整個語料庫的頻率
    # getcol(i).nonzero()得到一個特定的詞出現在哪幾個document
    # n 代表的是整個語料庫總共有多少個詞
    # 取倒數是因為之後要計算Importance時是隨著資料頻率增加而減少
    inverse_document_freqs.append( n / (data_counts.getcol(i).nonzero()[0].size))

tf_idf = []
tf_idf_index = []
term_in_document = count_vect.get_feature_names_out()
TF_IDF_mtx = data_counts.copy().toarray().astype(np.float64)
for i in range(len(term_frequencies)):
    temp = []
    for ind, freq in term_frequencies[i]:
        # 計算importance, 她隨著資料在句子中出現頻率增加而增加(data_counts.getcol(i).nonzero()[0].size)
        importance = inverse_document_freqs[ind] * freq
        TF_IDF_mtx[i, ind] = importance
        temp.append([ind, score])
    # 把importance由大到小排列
    temp.sort(key = lambda x: x[1], reverse = True)
    tf_idf_index.append(temp)

for index in tf_idf_index:
    #把index代表的字取出來
    index2Word = [[term_in_document[id[0]], id[1]] for id in index]
    tf_idf.append(index2Word)
TF_IDF = pd.DataFrame(TF_IDF_mtx.T, index = term_in_document, columns = data['Comment'])

data['TF_IDF'] = tf_idf
TF_IDF


5155


Comment,So there is no way for me to plug it in here in the US unless I go by a converter.,"Good case, Excellent value.",Great for the jawbone.,Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!,The mic is great.,I have to jiggle the plug to get it to line up right to get decent volume.,"If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.",If you are Razr owner...you must have this!,"Needless to say, I wasted my money.",What a waste of money and time!.,...,The refried beans that came with my meal were dried out and crusty and the food was bland.,Spend your money and time some place else.,A lady at the table next to us found a live green caterpillar In her salad.,the presentation of the food was awful.,I can't tell you how disappointed I was.,I think food should have flavor and texture and both were lacking.,Appetite instantly gone.,Overall I was not impressed and would not go back.,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.","Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing out the time it took to bring the check."
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z500a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zero,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zillion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
"""Bayes classifier"""
"""BernoulliNB()"""
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_fscore_support

score = data['Score'].values
dataset = data_counts.toarray() 
#切出 training用以及validation用
x_train, x_test, y_train, y_test = train_test_split(dataset, score, test_size=0.2)
bnb = BernoulliNB()
bnb.fit(x_train, y_train)
y_pred = bnb.predict(x_test)

precision,recall,f_score,_ = precision_recall_fscore_support(y_test, y_pred, average='micro')

print("precision: {:.3f},recall: {:.3f},f-score: {:.3f}".format(precision, recall, f_score))


"""MultinomialNB()"""
dataset2 = TF_IDF_mtx
x_train, x_test, y_train, y_test = train_test_split(dataset2, score, test_size=0.2)
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
predictResult = mnb.predict(x_test)


precision,recall,f_score,_ = precision_recall_fscore_support(y_test, y_pred, average='micro')

print("precision: {:.3f},recall: {:.3f},f-score: {:.3f}".format(precision, recall, f_score))

precision: 0.828,recall: 0.828,f-score: 0.828
precision: 0.495,recall: 0.495,f-score: 0.495
